In [210]:
from utils import  PkdbModel
from circos_utils import study_expand, bubbles_data,
from itertools import combinations, chain
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap
import numpy as np
from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.cluster import AgglomerativeClustering


%load_ext autoreload
%autoreload 2

In [211]:
# number of substance clsuters
n_cluster = 4

## Configuration for Circular Plots

This notebook creates the configuration files for the circular plots.

Tasks performed by the notebook:

 1. List Classifies substances by co-occurrence in studies. Derived substances are replaced but their parents already by the database. Label for a class is chosen to be most frequent substance within a class.
 2. Classifies studies into the same classes as substances. The most frequent class of substances used in a study is chosen to be the class of the study.
 3. Builds configuration files for circus plots for:
     1. Circular study plot with bars .  &rightarrow;  /circos/study/
     2. Circular study plot with bubbles.   &rightarrow;  /circos/study1/
     3. Circular plots with subset of study plots for each substance class. &rightarrow;  /circos/substance1/
     4. Circular substance plots with co-occurrence links and bubbles. &rightarrow;  /circos/study_caffeine/ ...
     
     
 

Loads the tabular data for studies and substances

In [212]:
# load data

studies = PkdbModel("studies", destination="1-preprocessed") 
substances = PkdbModel("substances", destination="1-preprocessed") 
substances.read()
studies.read()

In [206]:
# create substance combinations map
substance_combinations = iter(())
substances_all =[]
for study in studies.data.itertuples():
    if isinstance(study.substances,str):
        this_substance_combinations = study.substances.split(",")
        this_substance_combinations = [x.strip() for x in this_substance_combinations]
        substances_all.extend(this_substance_combinations)
        this_substance_combinations = iter(this_substance_combinations)
        substance_combinations = chain(substance_combinations,combinations(this_substance_combinations,2))
        
substances_combinations = pd.DataFrame(list(substance_combinations),columns=["substance1","substance2"])
substances_combinations["occurance"] = 1
coocurence_count = substances_combinations.groupby(["substance1","substance2"]).occurance.count().reset_index()
coocurence_count["substance1_id"] = coocurence_count["substance1"].apply(lambda x : substance_dict_rev[x])
coocurence_count["substance2_id"] = coocurence_count["substance2"].apply(lambda x : substance_dict_rev[x])

In [207]:
substances_combinations2 = pd.DataFrame()
substances_combinations2["substance1"] = substances_combinations["substance2"]
substances_combinations2["substance2"] = substances_combinations["substance1"]
substances_combinations2["occurance"] = substances_combinations["occurance"]

In [208]:
com = substances_combinations2.append(substances_combinations, ignore_index=True)
coocurence_both_count = com.groupby(["substance1","substance2"]).occurance.count().reset_index()
result = com.pivot_table(index="substance1", columns="substance2",aggfunc="sum")
res = result.fillna(0)

In [209]:
res

occurance                          \
substance2               (AFMU+1U+1X)/17U (mor+m3g+m6g)/(cod+c6g)   
substance1                                                          
(AFMU+1U+1X)/17U                      0.0                     0.0   
(mor+m3g+m6g)/(cod+c6g)               0.0                     0.0   
1-hydroxymidazolam                    0.0                     0.0   
17U                                   0.0                     0.0   
17U/px                                1.0                     0.0   
1U/(1U+1X)                            1.0                     0.0   
1U/1X                                 0.0                     0.0   
1X/caf                                0.0                     0.0   
1X/px                                 0.0                     0.0   
1X/tp                                 0.0                     0.0   
5-hydroxyomeprazole                   0.0                     0.0   
5home/ome                             0.0                     0.0   
6-hydroxychlorzoxazone                0.0                     0.0   
AAMU/(AAMU+1U+1X)                     0.0                     0.0   
FFA                                   0.0                     0.0   
GIP                                   0.0                     0.0   
GLP-1                                 0.0                     0.0   
[2-3H]glucose                         0.0                     0.0   
[3-OMG]glucose                        0.0                     0.0   
[6-3H]glucose                         0.0                     0.0   
[U-13C]glucose                        0.0                     0.0   
antipyrine                            0.0                     0.0   
c-peptide                             0.0                     0.0   
caf/px                                0.0                     0.0   
caffeine                              1.0                     0.0   
chlorzoxazone                         0.0                     0.0   
cimetidine                            0.0                     0.0   
ciprofloxacin                         0.0                     0.0   
cod/(mor+m3g+m6g+nmor)                0.0                     0.0   
cod/(ncod+ncg+nmor)                   0.0                     0.0   
...                                   ...                     ...   
morphine-6-glucuronide                0.0                     0.0   
norcodeine                            0.0                     0.0   
norcodeine-conjugates                 0.0                     0.0   
norcodeine-glucuronide                0.0                     0.0   
norepinephrine                        0.0                     0.0   
norfloxacin                           0.0                     0.0   
normorphine                           0.0                     0.0   
normorphine-glucuronide               0.0                     0.0   
ofloxacin                             0.0                     0.0   
omeprazole                            0.0                     0.0   
panadol extend                        0.0                     0.0   
paracetamol                           0.0                     0.0   
paracetamol cysteine                  0.0                     0.0   
paracetamol glucuronide               0.0                     0.0   
paracetamol mercapturate              0.0                     0.0   
paracetamol sulfate                   0.0                     0.0   
paraxanthine                          0.0                     0.0   
pipemidic acid                        0.0                     0.0   
propanolol                            0.0                     0.0   
px/caf                                0.0                     0.0   
quinidine                             0.0                     0.0   
spar/(2hspar+5hspar)                  0.0                     0.0   
sparteine                             0.0                     0.0   
sulfapyridine                         0.0                     0.0   
sulfasalazine                 

In [151]:
# normalize
x_trans = Normalizer(norm="l2").fit_transform(res)
X_name = pd.DataFrame(x_trans, index=res.index, columns=res.index)

In [152]:
# cluster substances based on coocurance
model = AgglomerativeClustering(linkage="single",n_clusters=4)
Y = model.fit_predict(X_name.T.corr())


In [153]:
#result


    

In [154]:
#substance to type 
substance_to_type = pd.DataFrame(Y, index=X_name.index, columns=["type"]).to_dict()["type"]

In [157]:
# type to label
name_type = substances.data.loc[substances.data.groupby("type")["study_number"].idxmax()][["name","type"]]
type_to_label = name_type.set_index("type").to_dict()["name"]
label_to_type = name_type.set_index("name").to_dict()["type"]

In [158]:
# label the all studies
studies.data["label"] = studies.data.apply(find_label,type_to_label=type_to_label, axis=1)
substances.data["label"] = substances.data.apply(find_label,type_to_label=type_to_label, axis=1)

In [159]:
studies.data[studies.data["name"]=="Baraka1990"][["name","label"]]

,name,label
105,Baraka1990,paracetamol


In [160]:
find_study_type(studies.data["substances"].loc[105],substance_to_type)

3

In [161]:
# circos datafile defines ids and length of area for ideograms

In [162]:
ideogram = pd.DataFrame()
ideogram["len"] = studies.data.groupby(["type"]).apply(len)
ideogram.index = ideogram.index.astype(int)
ideogram["label"] = ideogram.apply(lambda x: type_to_label[x.name], axis=1)

In [163]:
color_mapping = {}
#{"caffeine":"fill_color=red", "codeine":"fill_color=yellow","glucose":"fill_color=green","paracetamol":"fill_color=blue" }
with open("circos/study/data/studies.txt","w") as f:
    for idx, substance in ideogram.iterrows():
        
        number = idx + 1
        color_mapping[substance.label] = f"fill_color=spectral-5-div-{number}"
        f.write(f"chr - {substance.label} {substance.label} 0 {substance.len*3} spectral-5-div-{number}\n") 


In [164]:
color_mapping

{'caffeine': 'fill_color=spectral-5-div-1',
 'glucose': 'fill_color=spectral-5-div-2',
 'codeine': 'fill_color=spectral-5-div-3',
 'paracetamol': 'fill_color=spectral-5-div-4'}

In [165]:
studies_data = studies.data.sort_values(by=["label","name"])
normed_columns = ["groups_count","group_all_count","individuals_count","interventions_count", "outputs_count", "timecourses_count", "results_count"]
studies_data_norm = MaxAbsScaler().fit_transform(studies_data[normed_columns])
studies_data_norm = pd.DataFrame(studies_data_norm, index=studies_data.index, columns=normed_columns)

In [166]:
studies_data_norm

,groups_count,group_all_count,individuals_count,interventions_count,outputs_count,timecourses_count,results_count
2,0.176471,0.016636,0.409091,0.027027,0.098310,0.000000,0.088520
27,0.235294,0.025878,0.151515,0.027027,0.245776,0.045455,0.229599
25,0.294118,0.065619,0.530303,0.027027,0.215054,0.000000,0.193638
10,0.058824,0.011091,0.090909,0.027027,1.000000,0.545455,1.000000
35,0.411765,0.018484,0.303030,0.108108,0.230415,0.000000,0.207469
65,0.058824,0.005545,0.045455,0.081081,0.184332,0.000000,0.165975
32,0.176471,0.019409,0.159091,0.081081,0.390169,0.030303,0.356846
36,0.529412,0.040665,0.000000,0.027027,0.027650,0.000000,0.024896
28,0.176471,0.011091,0.000000,0.108108,0.050691,0.060606,0.056708
71,0.058824,0.011091,0.151515,0.108108,0.221198,0.045455,0.207469


In [167]:
studies_data_norm["label"] = studies_data["label"]
studies_data_norm.dropna(subset=["label"], inplace=True)
studies_data_norm["name"] = studies_data["name"]
studies_data_norm["ones"] = 1
studies_data_norm["fill_color"] = studies_data_norm.apply(lambda x: color_mapping[x["label"]], axis=1)
studies_data_norm["outputs_calculated_count"] = studies_data["outputs_calculated_count"]/studies_data["outputs_count"].max()

In [168]:
frames = []
for _ , data in studies_data_norm.groupby("label"):
    data["start"] = range(len(data)) 
    data["end"] = range(1,len(data)+1) 
    frames.append(data)
studies_data_norm = pd.concat(frames)


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [169]:
studies_data_norm[["label","start","end","name"]].to_csv("circos/study/data/study_names.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","group_all_count"]].to_csv("circos/study/data/all_subjects_number.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","individuals_count"]].to_csv("circos/study/data/individual_number.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","timecourses_count"]].to_csv("circos/study/data/timecourse_number.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","outputs_count"]].to_csv("circos/study/data/output_number.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","outputs_calculated_count"]].to_csv("circos/study/data/output_calculated_number.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","interventions_count"]].to_csv("circos/study/data/intervention_number.txt", sep=" ",header=False,index=False)
studies_data_norm[["label","start","end","ones","fill_color"]].to_csv("circos/study/data/substance_pie.txt", sep=" ",header=False,index=False)




# Circos Plot with circles

In [171]:
color_mapping = {}
#{"caffeine":"fill_color=red", "codeine":"fill_color=yellow","glucose":"fill_color=green","paracetamol":"fill_color=blue" }
with open("circos/study1/data/studies.txt","w") as f:
    for idx, substance in ideogram.iterrows():
        
        number = idx + 1
        color_mapping[substance.label] = f"fill_color=spectral-5-div-{number}"
        f.write(f"chr - {substance.label} {substance.label} 0 {substance.len*rows_per_study} pastel2-6-qual-{number}\n") 

In [172]:
frames = []
for _ , data in studies_data_norm.groupby("label"):
    
    
    data["start"] = range(0,len(data)*rows_per_study,rows_per_study) 
    data["end"] = range(rows_per_study,(len(data)+1)*rows_per_study,rows_per_study)
    frames.append(data)
studies_data_norm = pd.concat(frames)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [173]:
studies_data.dropna(subset=["label"], inplace=True)

studies_data[["start","end","ones","fill_color" ]] = studies_data_norm[["start","end","ones","fill_color"]]
studies_data["outputs_raw_count" ] = studies_data["outputs_count" ] - studies_data["outputs_calculated_count" ]
studies_data["group_not_individual" ] = studies_data["group_all_count" ] - studies_data["individuals_count" ]
studies_data["group_not_individual"] = studies_data["group_not_individual"].mask(studies_data["group_not_individual"]<0,0 )


In [174]:
studies_data[["label","start","end","name"]].to_csv("circos/study1/data/study_names.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","group_all_count"]].to_csv("circos/study1/data/all_subjects_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","group_all_count"]].to_csv("circos/study1/data/individual_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","timecourses_count"]].to_csv("circos/study1/data/timecourse_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","outputs_count"]].to_csv("circos/study1/data/output_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","outputs_calculated_count"]].to_csv("circos/study1/data/output_calculated_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","outputs_raw_count"]].to_csv("circos/study1/data/output_raw_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","interventions_count"]].to_csv("circos/study1/data/intervention_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","ones","fill_color"]].to_csv("circos/study1/data/substance_pie.txt", sep=" ",header=False,index=False)



SyntaxError: unexpected EOF while parsing (<ipython-input-204-c29714b096a0>, line 1)

In [176]:
divmod(134,50)

(2, 34)

In [178]:
outputs = pd.DataFrame()
timecourses = pd.DataFrame()
group_members = pd.DataFrame()
interventions = pd.DataFrame()
for idx,study in studies_data.iterrows():
#for idx,study in studies_data.iloc[::8,:].iterrows():
    
    raw_output = study_expand(study,"outputs_raw_count",25,"raw")
    if len(raw_output) > 0:
        outputs = outputs.append(raw_output)
        
    calc_output = study_expand(study,"outputs_calculated_count",25, "calc")
    if len(calc_output) > 0:
        outputs = outputs.append(calc_output)    
        
        
    timecourse = study_expand(study,"timecourses_count",25)
    if len(timecourse) > 0:
        timecourses = timecourses.append(timecourse)
    
    intervention = study_expand(study,"interventions_count",25)
    if len(intervention) > 0:
        interventions = interventions.append(intervention)
        
    
    group_subject = study_expand(study,"group_not_individual",25,"group_subject")
    if len(group_subject) > 0:
        group_members = group_members.append(group_subject)
        
    individual = study_expand(study,"individuals_count",25, "individuals")
    if len(individual) > 0:
        group_members = group_members.append(individual)    
        
    



In [179]:
group_members.groupby("circle_type").apply(len)

circle_type
circle_type=N     101
circle_type=n    1269
dtype: int64

In [180]:
#outputs = outputs[outputs["label"]== "codeine"]
#timecourses = timecourses[timecourses["label"]== "codeine"]
#group_members = group_members[group_members["label"]== "codeine"]
#interventions = interventions[interventions["label"]== "codeine"]


In [181]:
studies_data["group_all_count"] = studies_data["group_all_count"].astype(int)

In [182]:

outputs[["label","start","end","type","circle_type"]].to_csv("circos/study1/data/output_gly.txt", sep=" ",header=False,index=False)
timecourses[["label","start","end","type","circle_type"]].to_csv("circos/study1/data/timecourse_gly.txt", sep=" ",header=False,index=False)
group_members[["label","start","end","type","circle_type"]].to_csv("circos/study1/data/group_members_gly.txt", sep=" ",header=False,index=False)
interventions[["label","start","end","type","circle_type"]].to_csv("circos/study1/data/interventions_gly.txt", sep=" ",header=False,index=False)

In [183]:
studies_data[["label","start","end","group_all_count"]].to_csv("circos/study1/data/all_subjects_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","group_all_count"]].to_csv("circos/study1/data/individual_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","timecourses_count"]].to_csv("circos/study1/data/timecourse_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","outputs_count"]].to_csv("circos/study1/data/output_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","outputs_calculated_count"]].to_csv("circos/study1/data/output_calculated_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","outputs_raw_count"]].to_csv("circos/study1/data/output_raw_number.txt", sep=" ",header=False,index=False)

studies_data[["label","start","end","interventions_count"]].to_csv("circos/study1/data/intervention_number.txt", sep=" ",header=False,index=False)
studies_data[["label","start","end","ones","fill_color"]].to_csv("circos/study1/data/substance_pie.txt", sep=" ",header=False,index=False)

In [184]:
#circos_data_2["group_all_count_log"] = np.log(circos_data_2["group_all_count"])
#circos_data_2["group_all_count_norm"] = circos_data_2["group_all_count_log"]/circos_data_2["group_all_count_log"].max()

In [185]:
#circos_data_2[["label","start","end","group_all_count_norm"]].to_csv("circos/study/data/hist_amount_subjects.txt", sep=" ",header=False,index=False)

In [186]:
#circos_data_2["outputs_count_log"] = np.log(circos_data_2["outputs_count"])
#circos_data_2["outputs_count_norm"] = circos_data_2["outputs_count_log"]/circos_data_2["outputs_count_log"].max()
#circos_data_2[["label","start","end","outputs_count_norm"]].to_csv("circos/study/data/hist_amount_outputs.txt", sep=" ",header=False,index=False)

In [187]:
#circos_data_2["timecourses_count_log"] = np.log(circos_data_2["timecourses_count"])
#circos_data_2["timecourses_count_norm"] = circos_data_2["timecourses_count_log"]/circos_data_2["timecourses_count_log"].max()
#circos_data_2[["label","start","end","timecourses_count_norm"]].replace(-np.inf,0).to_csv("circos/study/data/hist_amount_timecourses.txt", sep=" ",header=False,index=False)


In [188]:
#circos_data_2[["label","start","end","timecourses_count_norm"]]

In [189]:
#color_mapping

In [190]:
#circos_data_2["ones"] = 1

In [191]:
#circos_data_2["fill_color"] = circos_data_2.apply(lambda x: color_mapping[x["label"]], axis=1)

In [192]:
#circos_data_2["fill_color"]

In [193]:
#circos_data_2[["label","start","end","ones","fill_color"]].to_csv("substance_pie.txt", sep=" ",header=False,index=False)


In [194]:
substance_subset = substances.data[["study_number","intervention_number","timecourse_number","output_number"]]
normed = MaxAbsScaler().fit_transform(substance_subset)
substances_normed = pd.DataFrame(normed, index=substance_subset.index, columns=substance_subset.columns)

In [195]:
substances_normed["name"] = substances.data["name"].apply(lambda x: x.replace(" ","_"))
substances_normed["label"] = substances.data["label"]
substances_normed["fill_color"] = substances_normed.apply(lambda x: color_mapping[x["label"]], axis=1)
substances_normed["ones"] = 1

In [196]:
frames = []
for substance, data in substances_normed.groupby("label"):
    data["start"] = range(len(data)) 
    data["end"] = range(1,len(data)+1) 
    frames.append(data)
substances_normed = pd.concat(frames)


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [197]:
len_labels = substances_normed.groupby("label").apply(len)
len_labels = pd.DataFrame(len_labels, columns=["substance_len"])
substance_categories = substances.data.merge(len_labels, left_on="name", right_on=len_labels.index)

In [198]:
with open("circos/substance/data/substances.txt","w") as f:
    i = 0
    for idx, substance in substance_categories.iterrows():
        number = i + 1
        f.write(f"chr - {substance.label} {substance.label} 0 {substance.substance_len} spectral-5-div-{number}\n") 

In [199]:
substances_normed[["label","start","end","ones","fill_color"]].to_csv("circos/substance/data/substance_pie.txt", sep=" ",header=False,index=False)
substances_normed[["label","start","end","study_number"]].to_csv("circos/substance/data/study_number.txt", sep=" ",header=False,index=False)
substances_normed[["label","start","end","intervention_number"]].to_csv("circos/substance/data/intervention_number.txt", sep=" ",header=False,index=False)
substances_normed[["label","start","end","output_number"]].to_csv("circos/substance/data/output_number.txt", sep=" ",header=False,index=False)
substances_normed[["label","start","end","timecourse_number"]].to_csv("circos/substance/data/timecourse_number.txt", sep=" ",header=False,index=False)
substances_normed[["label","start","end","name"]].to_csv("circos/substance/data/substance_name.txt", sep=" ",header=False,index=False)


In [200]:
substance_subset.max()

study_number             53
intervention_number      69
timecourse_number       161
output_number          4443
dtype: int64

In [201]:
frames = []
for substance, data in substances.data.groupby("label"):
    data["start"] = range(len(data)) 
    data["end"] = range(1,len(data)+1) 
    frames.append(data)
substances_data = pd.concat(frames)

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [202]:
substances_data

,name,study_number,intervention_number,timecourse_number,output_number,studies,interventions,outputs,timecourses,type,label,start,end
4,caffeine,53,69,161,4443,"['George1986', 'Bonati1982', 'Faber2004', 'Bal...","[2, 4, 10, 12, 14, 22, 28, 34, 36, 38, 44, 52,...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...","[7, 8, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98,...",0,caffeine,0,1
5,paraxanthine,16,3,46,290,"['Bonati1982', 'Harder1988', 'Culm-Merdek2005'...","[40, 42, 298]","[317, 318, 319, 320, 321, 322, 323, 324, 385, ...","[9, 10, 173, 179, 182, 184, 221, 222, 223, 224...",0,caffeine,1,2
13,px/caf,15,0,26,158,"['Granfors2005', 'Backman2008', 'Carrillo2000a...",[],"[269, 270, 271, 272, 273, 274, 275, 276, 277, ...","[11, 12, 174, 180, 295, 296, 404, 405, 406, 40...",0,caffeine,2,3
7,theobromine,7,1,22,74,"['Bonati1982', 'Culm-Merdek2005', 'McLean2002'...",[302],"[349, 350, 351, 352, 353, 354, 355, 356, 3886,...","[229, 230, 231, 232, 303, 311, 319, 327, 335, ...",0,caffeine,3,4
6,theophylline,7,1,22,78,"['Bonati1982', 'Culm-Merdek2005', 'McLean2002'...",[300],"[333, 334, 335, 336, 337, 338, 339, 340, 3830,...","[225, 226, 227, 228, 304, 312, 320, 328, 336, ...",0,caffeine,4,5
31,norfloxacin,4,10,0,0,"['Harder1988', 'Barnet1990', 'Carbo1989', 'Har...","[26, 70, 131, 132, 133, 134, 205, 206, 207, 208]",[],[],0,caffeine,5,6
40,pipemidic acid,4,10,0,0,"['Harder1988', 'Barnet1990', 'Carbo1989', 'Har...","[24, 72, 139, 140, 141, 142, 213, 214, 215, 216]",[],[],0,caffeine,6,7
33,ciprofloxacin,4,31,0,0,"['Harder1988', 'Harder1989', 'Stille1987', 'He...","[147, 148, 149, 150, 155, 156, 157, 158, 163, ...",[],[],0,caffeine,7,8
97,midazolam,4,4,3,28,"['Oh2012', 'Ghassabian2009', 'Turpault2009', '...","[112, 308, 340, 394]","[5334, 5335, 5336, 5337, 5338, 5339, 5340, 534...","[531, 593, 594]",0,caffeine,8,9
36,ofloxacin,3,12,0,0,"['Harder1988', 'Harder1989', 'Stille1987']","[123, 124, 125, 126, 197, 198, 199, 200, 363, ...",[],[],0,caffeine,9,10


In [203]:
interventions = pd.DataFrame()

for idx,substance in substances_data.iterrows():
    df_raw = substance[["label","start","end"]]
    df_raw["type"] = "raw"
    print(substance["intervention_number"])
    try:
        data = pd.concat([pd.DataFrame(df_raw).T]*substance["intervention_number"], ignore_index=True)
        outputs = outputs.append(data)
    except ValueError:
        pass
   

69
3
0
1
1
10
10
31
4
12
28
3
3
0
5
2
2
0
0
0
0
0
1
0
1
1
0
1
1
1
1
1
0
0
0
1
1
2
1
0
0
0
0
0
0
0
0
46
2
0
0
0
0
0
0
7
0


/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


0
0
0
0
0
2
1
0
0
0
0
0
0
1
1
0
0
2
13
1
0
5
0
0
0
0
0
0
1
0
0
0
0
0
0
0
20
0
0
0
0
1
1
